## Setup the local stack (for local testing)

In [ ]:
import syft_runtimes.high_low as syhl
from syft_rds.orchestra import setup_rds_server, remove_rds_stack_dir

In [ ]:
remove_rds_stack_dir(key="low_datasites", root_dir="./")

In [ ]:
DO = "do1@openmined.org"
do_stack_1 = setup_rds_server(email=DO, key="low_datasites", root_dir="./")

In [ ]:
lowside_client = do_stack_1.client
lowside_syftbox_dir = do_stack_1.client.workspace.data_dir
highside_syftbox_dir = lowside_syftbox_dir.parent / "high_datasites"
lowside_email = DO

In [ ]:
!rm -rf {highside_syftbox_dir}

In [ ]:
lowside_client.email

## DO (High): Initializing the high datasite

In [ ]:
# Low side settings
print(f"{lowside_syftbox_dir = }")
print(f"{highside_syftbox_dir = }")

In [ ]:
# High side settings
highlow_identifier = "highlow-1234"
highlow_identifier

In [ ]:
# First, initialize the high datasite - runs on the low side
highside_client: syhl.HighSideClient = syhl.initialize_high_datasite(
    email=DO,
    highlow_identifier=highlow_identifier,
    data_dir=highside_syftbox_dir,
    force_overwrite=True,
)

In [ ]:
highside_client.workspace.data_dir

## DO (high): Create a high-side dataset

In [ ]:
# Create some random mock and private data
from pathlib import Path
import random
import pandas as pd

num_mock_rows = 100
num_private_rows = 100

mock_data = {
    "age": [random.randint(0, 100) for _ in range(num_mock_rows)],
    "height": [random.uniform(150, 200) for _ in range(num_mock_rows)],
    "income": [random.randint(20000, 100000) for _ in range(num_mock_rows)],
}
private_data = {
    "age": [random.randint(0, 100) for _ in range(num_private_rows)],
    "height": [random.uniform(150, 200) for _ in range(num_private_rows)],
    "income": [random.randint(20000, 100000) for _ in range(num_private_rows)],
}

mock_df = pd.DataFrame(mock_data)
private_df = pd.DataFrame(private_data)

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)
mock_df.to_csv(data_dir / "mock_data.csv", index=False)
private_df.to_csv(data_dir / "private_data.csv", index=False)

In [ ]:
readme_content = """
# My example high-side dataset
"""
readme_path = data_dir / "README.md"
readme_path.write_text(readme_content)

In [ ]:
from syft_core.config import CONFIG_PATH_ENV
import os

# Set the high client as default syftbox client
os.environ[CONFIG_PATH_ENV] = str(highside_client.config_path)

In [ ]:
import syft_datasets as syd

DATASET_NAME = "my_cool_dataset"

highside_dataset = syd.create(
    name=DATASET_NAME,
    mock_path=data_dir / "mock_data.csv",
    private_path=data_dir / "private_data.csv",
    readme_path=readme_path,
    summary="This is a mock dataset for demonstration purposes.",
    tags=["example", "testing", "highside"],
    location=highlow_identifier,
)

In [ ]:
highside_dataset.mock_dir

In [ ]:
highside_dataset.private_dir

## DO (High): Connect to low-side

In [ ]:
lowside_client = highside_client.lowside_connect(
    highlow_identifier=highlow_identifier,
    lowside_data_dir=lowside_syftbox_dir,  # will be ssh_config in production
)

## DO (High): Rsync the dataset (mock part only) to the low side

In [ ]:
# Sync the dataset (mock part) to the low side
highside_client.sync_dataset(
    dataset_name=DATASET_NAME,
    verbose=True,
)

## DO (Low): Checks if the dataset exists (only the mock part)

In [ ]:
# We can now see the dataset on the low side!
from syft_datasets import SyftDatasetManager

lowside_dataset_manager = SyftDatasetManager(syftbox_client=lowside_client)
lowside_datasets = lowside_dataset_manager.get_all()

lowside_datasets

In [ ]:
lowside_datasets[0].mock_dir

In [ ]:
lowside_datasets[0].private_dir  # should raise an error

## DS: sees that the mock dataset is uploaded to the SyftBox network, play with it and submits a job to the DO's low datasite

In [ ]:
from syft_rds import init_session

ds_client = init_session(host=DO)

ds_client.job.submit(...)

## DO (High): Review and rsync pending jobs from the low side client

In [ ]:
highside_client.sync_pending_jobs()

## DO (High): Runs the job on private data


In [ ]:
# High side client runs the jobs using syft_runtimes
import syft_runtimes as syru

pending_jobs = syru.get_pending_jobs(client=highside_client)
for job in pending_jobs:
    highside_client.run_private(job)

## DO (High): Review and rsync job results in the `done` folder to the lowside client

In [ ]:
highside_client.sync_done_jobs()